### 1. Import Libraries

In [29]:
import yfinance as yf
from scipy.optimize import minimize
import numpy as np
import pandas as pd
from itertools import combinations

### 2. Data Obtention

In [30]:
# Lista de empresas a evaluar
tickers_list = ['VALE', 'BHP', 'RIO', 'SCCO', 'BTU', 'AMR', 'AES', 'UNP', 'CNI']

# Descargar datos históricos
tickers = yf.Tickers(" ".join(tickers_list))
hist = tickers.history(start='2021-01-01',end='2024-12-31')
adj_close = hist['Close'].dropna(axis=1, how='any')

[*********************100%***********************]  9 of 9 completed


### 3. Portfolios Analysis

In [31]:
# Función para calcular métricas del portafolio
def calcular_metricas(adj_close_values):
    R = np.log(adj_close_values[1:] / adj_close_values[:-1])  # Retornos logarítmicos
    RE = np.mean(R, axis=0) * 252  # Retorno esperado anualizado
    RI = np.std(R, axis=0) * np.sqrt(252)  # Riesgo anualizado
    S = np.cov(R, rowvar=False)  # Matriz de covarianza
    corr = np.corrcoef(R, rowvar=False)  # Matriz de correlación
    return RE, RI, S, corr

# Obtener métricas generales
adj_close_values = adj_close.values
RE, RI, S, correlation_matrix = calcular_metricas(adj_close_values)

# Evaluar todas las combinaciones posibles de 4 activos
mejores_portafolios = []
for subset in combinations(adj_close.columns, 4):
    indices = [adj_close.columns.get_loc(ticker) for ticker in subset]
    corr_submatrix = correlation_matrix[np.ix_(indices, indices)]

    # Filtrar combinaciones con correlación > 0.5
    if np.any(np.triu(corr_submatrix, k=1) > 0.5):
        continue  # Saltar esta combinación

    # Extraer datos de la combinación aceptable
    RE_sub = RE[indices]
    S_sub = S[np.ix_(indices, indices)]
    weights = np.ones(4) / 4  # Pesos iniciales iguales

    # Definir restricciones y límites
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    bounds = [(0, None)] * 4

    # Optimizar el portafolio
    res = minimize(lambda w: w @ S_sub @ w.T, x0=weights, method='SLSQP', bounds=bounds, constraints=constraints, options={'disp': False, 'maxiter': 1000, 'ftol': 1e-12})

    if res.success:
        # Calcular métricas del portafolio optimizado
        ReP = res.x @ RE_sub.T
        varP = res.x @ S_sub @ res.x.T
        RiP = np.sqrt(varP)*np.sqrt(252)
        SharpeP = ReP / RiP

        # Guardar resultados
        mejores_portafolios.append({
            "Activos": subset,
            "ReP (%)": round(ReP * 100, 4),
            "RiP (%)": round(RiP * 100, 4),
            "SharpeP": round(SharpeP, 4)
        })

# Ordenar por el mejor Sharpe Ratio
mejores_portafolios = sorted(mejores_portafolios, key=lambda x: x["SharpeP"], reverse=True)

# Convertir a DataFrame y mostrar los resultados
df_resultados = pd.DataFrame(mejores_portafolios)
print(df_resultados)

# Guardar en un archivo Excel
df_resultados.to_excel("mejores_portafolios.xlsx", index=False)
print("\nResultados guardados en 'mejores_portafolios.xlsx'")

                 Activos  ReP (%)  RiP (%)  SharpeP
0  (AES, BTU, SCCO, UNP)   3.6932  20.8851   0.1768
1   (AES, BTU, RIO, UNP)   2.1106  19.9808   0.1056
2   (AES, BHP, BTU, UNP)   1.3807  20.3460   0.0679
3  (AES, BTU, UNP, VALE)   0.5063  20.4585   0.0247
4  (AES, BTU, CNI, SCCO)  -0.8844  21.1696  -0.0418
5   (AES, BTU, CNI, RIO)  -1.8293  20.3709  -0.0898
6   (AES, BHP, BTU, CNI)  -2.6673  20.6931  -0.1289
7  (AES, BTU, CNI, VALE)  -3.2013  20.7785  -0.1541

Resultados guardados en 'mejores_portafolios.xlsx'
